In [87]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [88]:
df = pd.read_excel('Wartezeit Excel Neu!! 8.8.24.xlsx')

In [89]:
# trim all columns names
df.columns = df.columns.str.strip()
# strip all values
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

In [90]:
# set index to Identifationsnummer
df.set_index('Identifikationsnummer', inplace=True)

In [91]:
df.dtypes

Geburtsdatum          datetime64[ns]
Alter                        float64
Geschlecht                    object
Störungsgebiet                object
Anmeldedatum          datetime64[ns]
Diagnostiktermin      datetime64[ns]
Wartezeit in Tagen           float64
Montag                        object
Dienstag                      object
Mittwoch                      object
Donnerstag                    object
Freitag                       object
Versichert                    object
Verordnung                    object
Name                          object
ICD-Code                      object
Symptome                      object
Leitsymptomatik               object
Diagnosegruppe                object
dtype: object

In [92]:
# set Geburtsdatum as datetime format and calculate age in years
df['Geburtsdatum'] = pd.to_datetime(df['Geburtsdatum'])
df['Alter_Anmeldung'] = (df["Anmeldedatum"] - df['Geburtsdatum']).dt.days // 365
df['Alter_Diagnostiktermin'] = (df["Diagnostiktermin"] - df['Geburtsdatum']).dt.days // 365

In [93]:
# calculate Wartezeit in days
df['Wartezeit in Tagen'] = (df['Diagnostiktermin'] - df['Anmeldedatum']).dt.days

In [94]:
# check column Montag and create inner list with values of row
df["Montag"] = df["Montag"].apply(lambda x: x.split(', ') if pd.notna(x) else ["unverfügbar"])
df["Dienstag"] = df["Dienstag"].apply(lambda x: x.split(', ') if pd.notna(x) else ["unverfügbar"])
df["Mittwoch"] = df["Mittwoch"].apply(lambda x: x.split(', ') if pd.notna(x) else ["unverfügbar"])
df["Donnerstag"] = df["Donnerstag"].apply(lambda x: x.split(', ') if pd.notna(x) else ["unverfügbar"])
df["Freitag"] = df["Freitag"].apply(lambda x: x.split(', ') if pd.notna(x) else ["unverfügbar"])

In [95]:
df_dummies_montag = pd.get_dummies(df['Montag'].apply(pd.Series).stack(), prefix='Montag', dummy_na=False)
df_dummies_dienstag = pd.get_dummies(df['Dienstag'].apply(pd.Series).stack(), prefix='Dienstag', dummy_na=False)
df_dummies_mittwoch = pd.get_dummies(df['Mittwoch'].apply(pd.Series).stack(), prefix='Mittwoch', dummy_na=False)
df_dummies_donnerstag = pd.get_dummies(df['Donnerstag'].apply(pd.Series).stack(), prefix='Donnerstag', dummy_na=False)
df_dummies_freitag = pd.get_dummies(df['Freitag'].apply(pd.Series).stack(), prefix='Freitag', dummy_na=False)


In [96]:
df_dummies_montag_grouped = df_dummies_montag.groupby(level=0).sum()
df_dummies_dienstag_grouped = df_dummies_dienstag.groupby(level=0).sum()
df_dummies_mittwoch_grouped = df_dummies_mittwoch.groupby(level=0).sum()
df_dummies_donnerstag_grouped = df_dummies_donnerstag.groupby(level=0).sum()
df_dummies_freitag_grouped = df_dummies_freitag.groupby(level=0).sum()

In [97]:
df_dummies_versichert = pd.get_dummies(df['Versichert'], prefix='Versichert')
df_dummies_stoerungsgebiet = pd.get_dummies(df['Störungsgebiet'], prefix='Störungsgebiet')
df_dummies_geschlecht = pd.get_dummies(df['Geschlecht'], prefix='Geschlecht')
# df["Alter_Anmeldung_Bin"] = pd.cut(df["Alter_Anmeldung"], bins=[3, 7, 10, 12], labels=["4-6", "7-9", "10-12"])
# df_dummies_alter_anmeldung = pd.get_dummies(df['Alter_Anmeldung_Bin'], prefix='Alter_Anmeldung_Bin')
# df["Alter_Diagnostiktermin_Bin"] = pd.cut(df["Alter_Diagnostiktermin"], bins=[3, 7, 10, 12], labels=["4-6", "7-9", "10-12"])
# df_dummies_alter_diagnostiktermin = pd.get_dummies(df['Alter_Diagnostiktermin_Bin'], prefix='Alter_Diagnostiktermin_Bin')

In [98]:
# drop all columns that are used for dummies
# df.drop(['Geschlecht', 'Anmeldedatum', 'Diagnostiktermin', 'Störungsgebiet', 'Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Versichert', 'Verordnung', 'Unnamed: 15'], axis=1, inplace=True)

In [99]:
df.sort_index(inplace=True)

In [100]:
# merge all dummies
df = pd.concat([df, df_dummies_geschlecht, df_dummies_montag_grouped, df_dummies_dienstag_grouped, df_dummies_mittwoch_grouped, df_dummies_donnerstag_grouped, df_dummies_freitag_grouped, df_dummies_versichert, df_dummies_stoerungsgebiet], axis=1)

In [101]:
# df

In [102]:
# df = df.astype(int)

In [103]:
df["Geschlecht_männlich"] = df["Geschlecht_männlich"].astype(int)
df["Geschlecht_weiblich"] = df["Geschlecht_weiblich"].astype(int)
df["Versichert_gesetzlich"] = df["Versichert_gesetzlich"].astype(int)
df["Versichert_privat"] = df["Versichert_privat"].astype(int)
df["Störungsgebiet_OFD"] = df["Störungsgebiet_OFD"].astype(int)
df["Störungsgebiet_OFZ"] = df["Störungsgebiet_OFZ"].astype(int)
df["Störungsgebiet_RE1"] = df["Störungsgebiet_RE1"].astype(int)
df["Störungsgebiet_SCZ"] = df["Störungsgebiet_SCZ"].astype(int)
df["Störungsgebiet_SPZ"] = df["Störungsgebiet_SPZ"].astype(int)
df["Störungsgebiet_SP1"] = df["Störungsgebiet_SP1"].astype(int)
df["Störungsgebiet_SP3"] = df["Störungsgebiet_SP3"].astype(int)
df['Versichert_gesetzlich'] = df['Versichert_gesetzlich'].astype(int)
df['Versichert_privat'] = df['Versichert_privat'].astype(int)

In [104]:
# drop all columns that are used for dummies
df.drop(['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Verordnung'], axis=1, inplace=True)

In [105]:
# df.to_excel('df_final.xlsx')

In [106]:
# df.head()

In [107]:
# errechnen eines flexibilitätsscores anhand der tage an denen der patient kann
df['Flexibilitätsscore_Montag'] = df['Montag_morgens'] + df['Montag_vormittags'] + df['Montag_mittags'] + df['Montag_nachmittags'] + df['Montag_abends']
df['Flexibilitätsscore_Dienstag'] = df['Dienstag_morgens'] + df['Dienstag_vormittags'] + df['Dienstag_mittags'] + df['Dienstag_nachmittags'] + df['Dienstag_abends']
df['Flexibilitätsscore_Mittwoch'] = df['Mittwoch_morgens'] + df['Mittwoch_vormittags'] + df['Mittwoch_mittags'] + df['Mittwoch_nachmittags'] + df['Mittwoch_abends']
df['Flexibilitätsscore_Donnerstag'] = df['Donnerstag_morgens'] + df['Donnerstag_vormittags'] + df['Donnerstag_mittags'] + df['Donnerstag_nachmittags'] + df['Donnerstag_abends']
df['Flexibilitätsscore_Freitag'] = df['Freitag_morgens'] + df['Freitag_vormittags'] + df['Freitag_mittags'] + df['Freitag_nachmittags'] + df['Freitag_abends']
df['Flexibilitätsscore_Woche'] = df['Flexibilitätsscore_Montag'] + df['Flexibilitätsscore_Dienstag'] + df['Flexibilitätsscore_Mittwoch'] + df['Flexibilitätsscore_Donnerstag'] + df['Flexibilitätsscore_Freitag']

In [108]:
df.columns.tolist()

['Geburtsdatum',
 'Alter',
 'Geschlecht',
 'Störungsgebiet',
 'Anmeldedatum',
 'Diagnostiktermin',
 'Wartezeit in Tagen',
 'Versichert',
 'Name',
 'ICD-Code',
 'Symptome',
 'Leitsymptomatik',
 'Diagnosegruppe',
 'Alter_Anmeldung',
 'Alter_Diagnostiktermin',
 'Geschlecht_männlich',
 'Geschlecht_weiblich',
 'Montag_abends',
 'Montag_mittags',
 'Montag_morgens',
 'Montag_nachmittags',
 'Montag_unverfügbar',
 'Montag_vormittags',
 'Dienstag_abends',
 'Dienstag_mittags',
 'Dienstag_morgens',
 'Dienstag_nachmittags',
 'Dienstag_unverfügbar',
 'Dienstag_vormittags',
 'Mittwoch_abends',
 'Mittwoch_mittags',
 'Mittwoch_morgens',
 'Mittwoch_nachmittags',
 'Mittwoch_unverfügbar',
 'Mittwoch_vormittags',
 'Donnerstag_abends',
 'Donnerstag_mittags',
 'Donnerstag_morgens',
 'Donnerstag_nachmittags',
 'Donnerstag_unverfügbar',
 'Donnerstag_vormittags',
 'Freitag_abends',
 'Freitag_mittags',
 'Freitag_morgens',
 'Freitag_nachmittags',
 'Freitag_unverfügbar',
 'Freitag_vormittags',
 'Versichert_gesetzl

In [109]:
df.to_excel('df_final.xlsx')

In [110]:
# df.drop(columns=['Alter_12-18', 'Alter_18-67'], inplace=True)

In [111]:
corr = df.corr().round(3)
corr = corr.style.background_gradient(cmap='RdYlGn')

ValueError: could not convert string to float: 'weiblich'

In [ ]:
corr

,Geburtsdatum,Wartezeit in Tagen,Alter_Anmeldung,Alter_Diagnostiktermin,Geschlecht_männlich,Geschlecht_weiblich,Montag_abends,Montag_mittags,Montag_morgens,Montag_nachmittags,Montag_unverfügbar,Montag_vormittags,Dienstag_abends,Dienstag_mittags,Dienstag_morgens,Dienstag_nachmittags,Dienstag_unverfügbar,Dienstag_vormittags,Mittwoch_abends,Mittwoch_mittags,Mittwoch_morgens,Mittwoch_nachmittags,Mittwoch_unverfügbar,Mittwoch_vormittags,Donnerstag_abends,Donnerstag_mittags,Donnerstag_morgens,Donnerstag_nachmittags,Donnerstag_unverfügbar,Donnerstag_vormittags,Freitag_abends,Freitag_mittags,Freitag_morgens,Freitag_nachmittags,Freitag_unverfügbar,Freitag_vormittags,Versichert_gesetzlich,Versichert_privat,Störungsgebiet_OFD,Störungsgebiet_OFZ,Störungsgebiet_RE1,Störungsgebiet_SCZ,Störungsgebiet_SP1,Störungsgebiet_SP3,Störungsgebiet_SPZ
Geburtsdatum,1.000000,-0.179000,-0.939000,-0.941000,0.185000,-0.185000,-0.015000,0.149000,0.264000,0.149000,-0.022000,0.223000,0.076000,0.225000,0.321000,0.079000,-0.079000,0.288000,0.087000,0.212000,0.308000,0.019000,-0.048000,0.268000,-0.147000,0.179000,0.268000,-0.083000,0.103000,0.233000,-0.027000,0.231000,0.307000,-0.048000,0.013000,0.253000,0.155000,-0.155000,-0.258000,0.022000,-0.031000,-0.228000,0.167000,0.004000,-0.156000
Wartezeit in Tagen,-0.179000,1.000000,0.129000,0.214000,0.044000,-0.044000,-0.092000,-0.261000,-0.259000,-0.212000,0.134000,-0.206000,0.023000,-0.262000,-0.212000,-0.143000,0.001000,-0.225000,0.086000,-0.204000,-0.210000,-0.022000,-0.062000,-0.155000,0.166000,-0.223000,-0.237000,-0.048000,-0.124000,-0.220000,0.063000,-0.354000,-0.360000,0.009000,-0.080000,-0.295000,0.089000,-0.089000,0.036000,-0.030000,0.151000,0.188000,0.002000,-0.096000,-0.087000
Alter_Anmeldung,-0.939000,0.129000,1.000000,0.974000,-0.131000,0.131000,-0.025000,-0.222000,-0.278000,-0.163000,0.051000,-0.256000,-0.157000,-0.235000,-0.309000,-0.144000,0.150000,-0.278000,-0.048000,-0.258000,-0.321000,0.012000,0.045000,-0.299000,0.072000,-0.239000,-0.297000,-0.001000,-0.022000,-0.286000,0.044000,-0.292000,-0.337000,0.075000,0.004000,-0.301000,-0.124000,0.124000,0.294000,0.040000,-0.004000,0.209000,-0.237000,0.063000,0.177000
Alter_Diagnostiktermin,-0.941000,0.214000,0.974000,1.000000,-0.153000,0.153000,-0.026000,-0.232000,-0.300000,-0.135000,0.056000,-0.274000,-0.139000,-0.220000,-0.318000,-0.126000,0.135000,-0.300000,-0.048000,-0.252000,-0.327000,0.006000,0.034000,-0.300000,0.080000,-0.234000,-0.309000,0.024000,-0.049000,-0.290000,0.018000,-0.311000,-0.357000,0.061000,0.017000,-0.308000,-0.110000,0.110000,0.272000,0.029000,0.019000,0.191000,-0.221000,0.053000,0.157000
Geschlecht_männlich,0.185000,0.044000,-0.131000,-0.153000,1.000000,-1.000000,-0.031000,-0.026000,0.055000,0.052000,0.032000,0.028000,-0.102000,0.076000,-0.000000,-0.028000,0.140000,0.000000,0.153000,0.026000,-0.026000,0.055000,0.000000,-0.055000,-0.034000,0.026000,0.026000,-0.029000,0.040000,0.027000,0.031000,-0.025000,-0.110000,0.056000,-0.034000,-0.079000,0.140000,-0.140000,-0.113000,0.113000,0.229000,-0.113000,0.155000,-0.256000,0.000000
Geschlecht_weiblich,-0.185000,-0.044000,0.131000,0.153000,-1.000000,1.000000,0.031000,0.026000,-0.055000,-0.052000,-0.032000,-0.028000,0.102000,-0.076000,-0.000000,0.028000,-0.140000,0.000000,-0.153000,-0.026000,0.026000,-0.055000,0.000000,0.055000,0.034000,-0.026000,-0.026000,0.029000,-0.040000,-0.027000,-0.031000,0.025000,0.110000,-0.056000,0.034000,0.079000,-0.140000,0.140000,0.113000,-0.113000,-0.229000,0.113000,-0.155000,0.256000,0.000000
Montag_abends,-0.015000,-0.092000,-0.025000,-0.026000,-0.031000,0.031000,1.000000,0.306000,0.207000,0.708000,-0.925000,0.171000,0.103000,0.187000,0.189000,0.210000,-0.124000,0.207000,0.328000,0.201000,0.242000,0.260000,-0.381000,0.207000,0.185000,0.201000,0.201000,0.213000,-0.105000,0.218000,0.477000,0.249000,0.207000,0.364000,-0.351000,0.242000,0.039000,-0.039000,0.058000,0.058000,-0.161000,-0.217000,0.039000,0.032000,0.083000
Montag_mittags,0.149000,-0.261000,-0.222000,-0.

In [ ]:
import dataframe_image as dfi
dfi.export(corr, 'corr.png', max_cols=-1)